<a href="https://colab.research.google.com/github/vardhan2303A52384/Exp-AI/blob/main/exp_ai_ass_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb

In [2]:
data = pd.read_csv('/content/loan_approval.csv')
data.head()

,name,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,Allison Hill,East Jill,113810,389,39698,27,50.0,False
1,Brandon Hall,New Jamesside,44592,729,15446,28,55.0,False
2,Rhonda Smith,Lake Roberto,33278,584,11189,13,45.0,False
3,Gabrielle Davis,West Melanieview,127196,344,48823,29,50.0,False
4,Valerie Gray,Mariastad,66048,496,47174,4,25.0,False


In [3]:
data.columns

Index(['name', 'city', 'income', 'credit_score', 'loan_amount',
       'years_employed', 'points', 'loan_approved'],
      dtype='object')

In [4]:
data[data.select_dtypes(include='object').columns] = data.select_dtypes(include='object').apply(lambda x: x.str.strip())

In [5]:
data['loan_approved'].unique()

array([False,  True])

In [6]:
data = data.drop(columns=["name"])

In [7]:
# Inspect shape
print("Shape:", data.shape)

Shape: (2000, 7)


In [8]:
# Inspect datatypes
print("\nData types:\n", data.dtypes)


Data types:
 city               object
income              int64
credit_score        int64
loan_amount         int64
years_employed      int64
points            float64
loan_approved        bool
dtype: object


In [9]:
# Check missing values
print("\nMissing values:\n", data.isnull().sum())


Missing values:
 city              0
income            0
credit_score      0
loan_amount       0
years_employed    0
points            0
loan_approved     0
dtype: int64


In [10]:
print(data['loan_approved'].value_counts())

loan_approved
False    1121
True      879
Name: count, dtype: int64


In [11]:
for col in data.select_dtypes(include="number").columns:
    data[col].fillna(data[col].median(), inplace=True)
for col in data.select_dtypes(include="object").columns:
    data[col].fillna(data[col].mode()[0], inplace=True)

/tmp/ipython-input-531700803.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
/tmp/ipython-input-531700803.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [12]:
# Encode categorical columns
le = LabelEncoder()
for col in data.select_dtypes(include="object").columns:
    data[col] = le.fit_transform(data[col])

In [13]:
# Features and target
X = data.drop("loan_approved", axis=1)
y = data["loan_approved"]

In [14]:
# Scale numeric features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [15]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [16]:
# Handle imbalance with SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [17]:
print(y_train.value_counts())       # before SMOTE
print(y_train_res.value_counts())   # after SMOTE

loan_approved
False    897
True     703
Name: count, dtype: int64
loan_approved
False    897
True     897
Name: count, dtype: int64


In [18]:
# Logistic Regression
lr = LogisticRegression(class_weight="balanced", random_state=42)
lr.fit(X_train_res, y_train_res)

# Random Forest
rf = RandomForestClassifier(class_weight="balanced", random_state=42)
rf.fit(X_train_res, y_train_res)

# XGBoost
xgb_model = xgb.XGBClassifier(scale_pos_weight=(y_train_res.value_counts()[0]/y_train_res.value_counts()[1]), random_state=42, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train_res, y_train_res)

/tmp/ipython-input-4094232208.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  xgb_model = xgb.XGBClassifier(scale_pos_weight=(y_train_res.value_counts()[0]/y_train_res.value_counts()[1]), random_state=42, use_label_encoder=False, eval_metric="logloss")
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [17:26:16] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [19]:
#Evaluate models
models = {"Logistic Regression": lr, "Random Forest": rf, "XGBoost": xgb_model}

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{name}")
    print(classification_report(y_test, y_pred))
    print("\n")

Logistic Regression
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       224
        True       1.00      1.00      1.00       176

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



Random Forest
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       224
        True       1.00      1.00      1.00       176

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



XGBoost
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       224
        True       1.00      1.00      1.00       176

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00     

In [20]:
!pip install dice-ml --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.4 MB/s eta 0:00:00


In [21]:
import dice_ml
from dice_ml.utils import helpers

In [22]:
# 1. Clean column names (already done in previous steps)

# 2. Encode categorical columns only
le = LabelEncoder()
for col in ['city']: # Encode 'city' column
    data[col] = le.fit_transform(data[col])

# 3. Split features and target (unscaled)
X = data.drop("loan_approved", axis=1) # Use 'loan_approved' as target
y = data["loan_approved"] # Use 'loan_approved' as target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 4. SMOTE (optional)
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# 5. Train Random Forest on **same features used in DiCE**
rf = RandomForestClassifier(class_weight="balanced", random_state=42)
rf.fit(X_train_res, y_train_res)

# 6. Now DiCE query instance from X_test will match feature names exactly
query_instance = X_test[y_test==0].iloc[[0]]  # first rejected

In [23]:
# Imports
import dice_ml
import pandas as pd

# 1. Fix column spaces (already done in previous steps)

# 2. Define continuous features
continuous_features = [
    'income', 'credit_score', 'loan_amount',
    'years_employed', 'points'
]

# 3. Create DiCE Data object (original, unscaled)
d = dice_ml.Data(
    dataframe=data,
    continuous_features=continuous_features,
    outcome_name='loan_approved'
)

# 4. Create DiCE Model object using trained Random Forest
m = dice_ml.Model(model=rf, backend='sklearn')

# 5. Initialize DiCE explainer
exp = dice_ml.Dice(d, m)

# 6. Pick a test instance predicted as negative (Rejected)
#    Use original unscaled data and drop target column
neg_idx = y_test[y_test==0].index[0]  # first Rejected
query_instance = data.loc[[neg_idx]].drop(columns=['loan_approved'])  # only features

# 7. Generate 3 counterfactuals
cf_examples = exp.generate_counterfactuals(
    query_instance,
    total_CFs=3,
    desired_class="opposite"
)

# 8. Visualize counterfactuals, showing only changed features
cf_examples.visualize_as_dataframe(show_only_changes=True)
cf_df = cf_examples.cf_examples_list[0].final_cfs_df
print("Counterfactuals vs Original:\n", cf_df)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1790' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
100%|██████████| 1/1 [00:00<00:00,  3.56it/s]

Query instance (original outcome : False)


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,234,62662,466,26460,25,35.0,False



Diverse Counterfactual set (new outcome: True)


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,-,-,-,-,-,94.4,True
1,-,-,-,-,-,71.5,True
2,-,-,-,-,-,98.1,True


Counterfactuals vs Original:
    city  income  credit_score  loan_amount  years_employed  points  \
0   234   62662           466        26460              25    94.4   
1   234   62662           466        43700              25    71.5   
2   234   62662           466        26460              25    98.1   

   loan_approved  
0           True  
1           True  
2           True  


In [24]:
query_instance_reindexed = query_instance.iloc[0].reindex(cf_df.columns)
changed_features = cf_df.loc[:, (cf_df != query_instance_reindexed).any()]
influential_features = changed_features.columns.tolist()
print("Most influential features:", influential_features)

Most influential features: ['loan_amount', 'points', 'loan_approved']


In [25]:
# 8. Check realism/actionability
print("Original instance:\n", query_instance)
print("Counterfactuals:\n", cf_df)

Original instance:
      city  income  credit_score  loan_amount  years_employed  points
795   234   62662           466        26460              25    35.0
Counterfactuals:
    city  income  credit_score  loan_amount  years_employed  points  \
0   234   62662           466        26460              25    94.4   
1   234   62662           466        43700              25    71.5   
2   234   62662           466        26460              25    98.1   

   loan_approved  
0           True  
1           True  
2           True  


In [26]:
# 9. Generate counterfactuals using Manhattan distance
cf_examples_manhattan = exp.generate_counterfactuals(
    query_instance, total_CFs=3, desired_class="opposite"
)
cf_examples_manhattan.visualize_as_dataframe(show_only_changes=True)
cf_df_manhattan = cf_examples_manhattan.cf_examples_list[0].final_cfs_df
print("Counterfactuals with Manhattan distance:\n", cf_df_manhattan)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '665' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
100%|██████████| 1/1 [00:00<00:00,  4.99it/s]

Query instance (original outcome : False)


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,234,62662,466,26460,25,35.0,False



Diverse Counterfactual set (new outcome: True)


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,-,-,-,-,-,59.1,True
1,457,-,-,-,-,63.7,True
2,-,-,-,-,-,94.8,True


Counterfactuals with Manhattan distance:
    city  income  credit_score  loan_amount  years_employed  points  \
0   234   62662           463        26460              25    59.1   
1   457   62662           466        26460              25    63.7   
2   234   62662           466        26460              25    94.8   

   loan_approved  
0           True  
1           True  
2           True  
